In [ ]:
import openai
import langchain
import pinecone
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import os

In [ ]:
## reading doc
def read_doc(directory):
    file_loader = PyPDFDirectoryLoader(directory)
    documents = file_loader.load()
    return documents

In [ ]:
doc = read_doc('documents/')

In [ ]:
doc

In [ ]:
len(doc)

In [ ]:
## dividing into chunks
def chunk_data(docs, chunk_size = 800, chunk_overlap = 50):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = chunk_size, chunk_overlap = chunk_overlap)
    doc = text_splitter.split_documents(docs)
    return docs


In [ ]:
documents = chunk_data(docs = doc)
documents


In [ ]:
len(documents)

In [ ]:
#Embedding technique of OPENAI
embeddings = OpenAIEmbeddings(api_key = os.environ['OPENAI_API_KEY'])
embeddings

In [ ]:
vectors = embeddings.embed_query('how are you?')
len(vectors)

In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(
    api_key=os.environ.get("PINECONE_API_KEY")
)

# Now do stuff
index_name = 'langchainvector'
if 'langchainvector' not in pc.list_indexes().names():
    pc.create_index(
        name=index_name, 
        dimension=1536, 
        metric='euclidean',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-west-2'
        )
    )

In [ ]:
from langchain.vectorstores import Pinecone as PineconeStore
index = PineconeStore.from_documents(doc, embeddings, index_name = index_name)

In [ ]:
# Cosine Similarity Retrieve from VectorDB
def retrieve_query(query, k = 2):
    matching_results = index.similarity_search(query, k = 2)
    return matching_results

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI

In [ ]:
llm = OpenAI(model_name = 'davinci-002', temperature = 0.5)
chain = load_qa_chain(llm, chain_type = 'stuff')

In [ ]:
#Search answers from Vector DB
def retrieve_answers(query):
    doc_search = retrieve_query(query)
    print(doc_search)
    response = chain.run(input_documents = doc_search, question = query)
    return response

In [ ]:
our_query = 'our_query'
answer = retrieve_answers(our_query)
print(answer)